In [88]:
import pandas as pd
import numpy as np
import re

### Import datasets

In [89]:
tweets1 = pd.read_json("BLM_1.json")
tweets2 = pd.read_json("BLM_2.json")
tweets3 = pd.read_json("BLM_3.json")
tweets = tweets1.append([tweets2, tweets3])
tweets = tweets.sort_values(by='created_at')

In [90]:
tweets = tweets.reset_index(drop=True)
tweets['long'] = 0.
tweets['lat'] = 0.
for idx, val in enumerate(tweets['coordinates']):
        tweets['long'][idx] = val['coordinates'][0]
        tweets['lat'][idx] = val['coordinates'][1]
tweets = tweets.drop(columns=['coordinates'])

### From here we get lemmatize the text

In [91]:
def clean_hyperlinks(text):
    result = re.sub(r'https?://[^\s<>"]+|www\.[^\s<>"]+'," ", text)
    return result

In [92]:
#tweets = tweets[tweets['lang'] == 'en']
tweets['text'] = tweets['text'].apply(clean_hyperlinks)
#tweets['text'].replace('', np.nan, inplace=True)
#tweets.dropna(subset=['text'], inplace=True)

In [93]:
def string_manipulation(df,column): 
    #extract hashtags
    df["hashtag"]  = df[column].str.findall(r'#.*?(?=\s|$)')
    #extract twitter account references
    df["accounts"] = df[column].str.findall(r'@.*?(?=\s|$)')
    return df
tweets = string_manipulation(tweets,"text")

In [94]:
tweets['hashtag'] = tweets['hashtag'].apply(lambda x: ','.join(map(str, x)))
from string import punctuation
def punctation(text): 
    result = text.replace("#", "")
    result = result.lower()
    return result
tweets['hashtag'] = tweets['hashtag'].apply(punctation)

In [95]:
def inlist(text): 
    result = text.split(",")
    return result
tweets['hashtag'] = tweets['hashtag'].apply(inlist)

In [96]:
list_of_tweets = []
for tweet in tweets['text']:
    temp = tweet
    list_of_tweets.append(temp)

In [97]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

# Create a set of stopwords
stop = set(stopwords.words('english'))

# Create a set of punctuation words 
exclude = set(string.punctuation) 

# This is the function makeing the lemmatization
lemma = WordNetLemmatizer()

# In this function we perform the entire cleaning
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

# This is the clean corpus.
doc_clean = [clean(doc).split() for doc in list_of_tweets] 
tweets['token'] = doc_clean

## make date correct

In [98]:
for idx, val in enumerate(tweets['created_at']):
    tweets['created_at'][idx] = str(tweets['created_at'][idx]).split()[0]

### import protest data


In [99]:
protest = pd.read_csv('/Users/sam/Documents/GitHub/DSP_B6/Protests_BLM_LongLat_210104.csv')
protest['protest'] = 1
protest = protest.rename(columns={"EVENT_DATE": "created_at", "LATITUDE": "lat", 'LONGITUDE': 'long'})
protest['long'] = protest['long'].round(3)
protest['lat'] = protest['lat'].round(3)

In [100]:
tweets['long'] = tweets['long'].round(3)
tweets['lat'] = tweets['lat'].round(3)

In [105]:
merged = pd.merge(tweets, protest,  how='left', on=['created_at','long','lat'], keep='first')
merged = merged.fillna(0)

TypeError: merge() got an unexpected keyword argument 'keep'

In [103]:
print(len(tweets))
print(len(protest))
print(len(merged))

11685
7184
11689


In [102]:
sum(merged['protest'])

182.0

In [85]:
#tweets t0 text file
tweets.to_csv('tweets_BLM.csv')
#to excel (better for tableau)
tweets.to_excel("tweets_BLM.xlsx")